# Naive Bayes - 20news
In this exercise we use the naive Bayes method for text classification. 
In the [accompanying notebook](./HW1_2IIG0_24_25.ipynb), the `20newsgroups` dataset is loaded for four classes of newsgroups: `rec.autos`, `rec.motorcycles`, `rec.sport.baseball`, `rec.sport.hockey`. 
The text documents are transformed to a bag of words representation, given by a data matrix $D∈{0,1}^{n×d}$ where each row represents a document and every column a word. $D$ is an indicator matrix of the words that occur in each document.

In [1]:
import numpy as np
import sklearn

In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

## Exercise 5a
Compute the class prior probabilities $p(y)$:
* $p(y = 0)$
* $p(y = 1)$
* $p(y = 2)$
* $p(y = 3)$

## Exercise 5b
What are the log-probabilities of the word 'naive' given each class? Use Laplace smoothing with $\alpha=1e−5$. Note that the log is in ML as a default the natural logarithm to the base of $e$.
Assuming that $x_{naive}$ denotes the random variable for the feature-word 'naive', compute the following probabilities:
* $\log p(x_{naive}=1∣y=0)$
* $\log p(x_{naive}=1∣y=1)$
* $\log p(x_{naive}=1∣y=2)$

## Exercise 5c
What is the posterior probability that a document belongs to the classes `rec.autos`, `rec.motorcycles`, `rec.sport.baseball`, or `rec.sport.hockey`, given that the words 'autos', 'motorcycles', 'baseball', or 'hockey' respectively appear in the document? 
Use Bayes' theorem to compute the posterior probability for each of the following:
* $p(y=0∣x_{auto}=1)$
* $p(y=1∣x_{motorcycles}=1)$
* $p(y=2∣x_{baseball}=1)$
* $p(y=3∣x_{hockey}=1)$